### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
import os

# File to Load (
file_to_load = os.path.join(".","Resources","purchase_data.csv")

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)

## Player Count

* Display the total number of players


In [2]:
#Total number of players is the total number of unique values in column "SN"
print(f'Total number of players: {len(purchase_data["SN"].unique())}')
purchase_data.head()

Total number of players: 576


,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [60]:
unique_items = len(purchase_data["Item ID"].unique())
average_price = purchase_data["Price"].mean()
purchases = len(purchase_data["Purchase ID"].unique())
total_revenue = purchase_data["Price"].sum()
summary_df = pd.DataFrame({'Number of Unique Items': [unique_items], 'Average Price': [average_price],'Number of Purchases': [purchases],
                 'Total Revenue': [total_revenue]})
cleaned_df = summary_df.style.format({'Average Price':'${0:,.2f}','Total Revenue':'${0:,.2f}'})
cleaned_df

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,"$2,379.77"


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [5]:
summary_df = purchase_data.drop_duplicates(subset='SN').groupby(['Gender']).count().reset_index()
summary_df = summary_df[['Gender','SN']].sort_values(by='SN',ascending=False)
total = len(purchase_data["SN"].unique())
summary_df["Percentage"] = pd.to_numeric(summary_df['SN'])/total
summary_df = summary_df.rename(columns={'SN': 'Total Count'})
output = summary_df.style.format({'Percentage':'{:.2%}'})
output = output.hide_index()
output

Gender,Total Count,Percentage
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%



## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [141]:
#Get dataframes for count, sum and average, and unique people format them, rename them and merge them, calculate the average per person and format the output
count_df= purchase_data.groupby(['Gender']).count().reset_index()
count_df = count_df[['Gender','Purchase ID']]
count_df.rename(columns={'Purchase ID': 'Purchase Count'},inplace=True)
sum_df = purchase_data.groupby(['Gender']).sum().reset_index()
sum_df = sum_df[['Gender','Price']]
sum_df.rename(columns={'Price': 'Total Purchase Value'},inplace=True)
mean_df = purchase_data.groupby(['Gender']).mean().reset_index()
mean_df = mean_df[['Gender','Price']]
mean_df.rename(columns={'Price': 'Average Purchase Price'},inplace=True)
average_person_df = purchase_data.drop_duplicates(subset='SN').groupby(['Gender']).count().reset_index()
average_person_df = average_person_df[['Gender','SN']]
average_person_df.rename(columns={'SN': 'Total people'},inplace=True)
summary_df = ((count_df.merge(mean_df,on='Gender')).merge(sum_df,on='Gender')).merge(average_person_df,on='Gender')
summary_df['Average Total Purchases per Person'] = summary_df['Total Purchase Value']/summary_df['Total people']
summary_df = summary_df.drop(['Total people'],axis=1)
summary_df.set_index('Gender',inplace=True)
cleaned_df = summary_df.style.format({'Average Purchase Price':'${0:,.2f}','Total Purchase Value':'${0:,.2f}','Average Total Purchases per Person':'${0:,.2f}'})
cleaned_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Average Total Purchases per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [142]:
age_df = purchase_data.drop_duplicates(subset='SN')['Age']
total_players = len(age_df)
bins = [0,9,14,19,24,29,34,39,100]
labels = ['<10','10-14','15-19','20-24','25-29','30-34','35-39','40+']
age_bins = pd.cut(age_df, bins,labels=labels,include_lowest=True,right=True,ordered=True)
age_bins = pd.DataFrame(age_bins.groupby(age_bins).count())
age_bins.rename(columns={'Age':'Total Count'},inplace=True)
age_bins.reset_index(inplace=True)
age_bins['Percentage of Players'] = age_bins['Total Count']/total_players
for_later= age_bins
cleaned_df = age_bins.style.format({'Percentage of Players':'{:.2%}'})
cleaned_df

,Age,Total Count,Percentage of Players
0,<10,17,2.95%
1,10-14,22,3.82%
2,15-19,107,18.58%
3,20-24,258,44.79%
4,25-29,77,13.37%
5,30-34,52,9.03%
6,35-39,31,5.38%
7,40+,12,2.08%


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [161]:
bins = [0,9,14,19,24,29,34,39,100]
labels = ['<10','10-14','15-19','20-24','25-29','30-34','35-39','40+']
age_bins = purchase_data
age_bins['Bin']=pd.cut(purchase_data['Age'], bins,labels=labels,include_lowest=True,right=True,ordered=True)
count_df = age_bins.groupby(['Bin']).count().reset_index()
count_df = count_df[['Bin','Purchase ID']]
count_df.rename(columns={'Purchase ID': 'Purchase Count'},inplace=True)
mean_df = age_bins.groupby(['Bin']).mean().reset_index()
mean_df = mean_df[['Bin','Price']]
mean_df.rename(columns={'Price': 'Average Purchase Price'},inplace=True)

sum_df = age_bins.groupby(['Bin']).sum().reset_index()
sum_df = sum_df[['Bin','Price']]
sum_df.rename(columns={'Price': 'Total Purchase Value'},inplace=True)
#variable is from an earlier code block
unique_bins= for_later
unique_bins = unique_bins[['Age','Total Count']]
unique_bins = unique_bins.rename(columns={'Age':'Bin'})
summary_df = count_df.merge(mean_df,on='Bin')
summary_df = summary_df.merge(sum_df,on='Bin')
summary_df = summary_df.merge(unique_bins,how='left',on='Bin')
summary_df['Avg Total Purchase per Person']=summary_df['Total Purchase Value']/summary_df['Total Count']
summary_df = summary_df.drop(columns=['Total Count'])
cleaned_df = summary_df.style.format({'Average Purchase Price':'${0:,.2f}','Total Purchase Value':'${0:,.2f}','Avg Total Purchase per Person':'${0:,.2f}'})
cleaned_df



,Bin,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
0,<10,23,$3.35,$77.13,$4.54
1,10-14,28,$2.96,$82.78,$3.76
2,15-19,136,$3.04,$412.89,$3.86
3,20-24,365,$3.05,"$1,114.06",$4.32
4,25-29,101,$2.90,$293.00,$3.81
5,30-34,73,$2.93,$214.00,$4.12
6,35-39,41,$3.60,$147.67,$4.76
7,40+,13,$2.94,$38.24,$3.19


## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [173]:
count_df= purchase_data.groupby(['SN']).count().reset_index()
count_df = count_df[['SN','Purchase ID']]
count_df.rename(columns={'Purchase ID': 'Purchase Count'},inplace=True)
sum_df = purchase_data.groupby(['SN']).sum().reset_index()
sum_df = sum_df[['SN','Price']]
sum_df.rename(columns={'Price': 'Total Purchase Value'},inplace=True)
mean_df = purchase_data.groupby(['SN']).mean().reset_index()
mean_df = mean_df[['SN','Price']]
mean_df.rename(columns={'Price': 'Average Purchase Price'},inplace=True)
merged_df = count_df.merge(mean_df,on='SN')
merged_df = merged_df.merge(sum_df,on='SN')
merged_df = merged_df.sort_values(by=['Total Purchase Value'],ascending=False)
cleaned_data = merged_df.head(5).style.format({'Average Purchase Price':'${0:,.2f}','Total Purchase Value':'${0:,.2f}'})
cleaned_data

,SN,Purchase Count,Average Purchase Price,Total Purchase Value
360,Lisosia93,5,$3.79,$18.96
246,Idastidru52,4,$3.86,$15.45
106,Chamjask73,3,$4.61,$13.83
275,Iral74,4,$3.40,$13.62
281,Iskadarya95,3,$4.37,$13.10


## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, average item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [194]:
items_df=purchase_data[['Item ID','Item Name','Price']]
no_dupes = items_df.drop_duplicates(subset=['Item ID'])
count_df= items_df.groupby(['Item ID']).count().reset_index()
count_df = count_df[['Item ID','Price']]
count_df.rename(columns={'Price': 'Purchase Count'},inplace=True)
sum_df= items_df.groupby(['Item ID']).sum().reset_index()
sum_df = sum_df[['Item ID','Price']]
sum_df.rename(columns={'Price': 'Total Purchase Value'},inplace=True)
merged_df = no_dupes.merge(count_df,on='Item ID')
merged_df = merged_df.merge(sum_df,on='Item ID')
merged_df = merged_df.sort_values(by=['Purchase Count'],ascending=False)
cleaned_data = merged_df.head(5).style.format({'Price':'${0:,.2f}','Total Purchase Value':'${0:,.2f}'})
cleaned_data

,Item ID,Item Name,Price,Purchase Count,Total Purchase Value
2,92,Final Critic,$4.88,13,$59.99
24,178,"Oathbreaker, Last Hope of the Breaking Storm",$4.23,12,$50.76
0,108,"Extraction, Quickblade Of Trembling Hands",$3.53,9,$31.77
17,82,Nirvana,$4.90,9,$44.10
93,145,Fiery Glass Crusader,$4.58,9,$41.22


## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame



In [196]:
items_df=purchase_data[['Item ID','Item Name','Price']]
no_dupes = items_df.drop_duplicates(subset=['Item ID'])
count_df= items_df.groupby(['Item ID']).count().reset_index()
count_df = count_df[['Item ID','Price']]
count_df.rename(columns={'Price': 'Purchase Count'},inplace=True)
sum_df= items_df.groupby(['Item ID']).sum().reset_index()
sum_df = sum_df[['Item ID','Price']]
sum_df.rename(columns={'Price': 'Total Purchase Value'},inplace=True)
merged_df = no_dupes.merge(count_df,on='Item ID')
merged_df = merged_df.merge(sum_df,on='Item ID')
merged_df = merged_df.sort_values(by=['Total Purchase Value'],ascending=False)
cleaned_data = merged_df.head(5).style.format({'Price':'${0:,.2f}','Total Purchase Value':'${0:,.2f}'})
cleaned_data

,Item ID,Item Name,Price,Purchase Count,Total Purchase Value
2,92,Final Critic,$4.88,13,$59.99
24,178,"Oathbreaker, Last Hope of the Breaking Storm",$4.23,12,$50.76
17,82,Nirvana,$4.90,9,$44.10
93,145,Fiery Glass Crusader,$4.58,9,$41.22
111,103,Singed Scalpel,$4.35,8,$34.80
